In [22]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100.0%


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




4.4%

100.0%
100.0%


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [23]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


Using mps device


In [37]:
# add a custom layer
class CustomDense(nn.Module):

    def __init__(self, size_in, size_out):
        super().__init__()

        self.weights = nn.Parameter(
            torch.Tensor(size_out, size_in)
        )  
        # nn.Parameter - trainable parameters, it's what we send to the constructor of the Optimizier.
        # serach for model.parameters() 

        self.bias = nn.Parameter(
            torch.Tensor(size_out)
        ) 

        # initialize weights and biases

        # He initialization
        nn.init.nn.init.kaiming_uniform_(self.weights, a=np.sqrt(5))
        nn.init.uniform(self.bias, -0.1, 0.1)

    def forward(self, x):
        w_times_x= torch.mm(x, self.weights.t())
        return torch.add(w_times_x, self.bias)  # w times x + b

In [36]:

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            CustomDense(28*28, 512), # instead of nn.Linear
            nn.ReLU(),
            CustomDense(512, 512),
            nn.ReLU(),
            CustomDense(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

#model = torch.compile(model)

AttributeError: module 'torch.nn.init' has no attribute 'nn'

In [32]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [33]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    # make it faster, no gradient computation
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [34]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")



Epoch 1
-------------------------------
loss: 2.297671  [   64/60000]
loss: 2.288376  [ 6464/60000]
loss: 2.269763  [12864/60000]
loss: 2.265694  [19264/60000]
loss: 2.253272  [25664/60000]
loss: 2.223988  [32064/60000]
loss: 2.235089  [38464/60000]
loss: 2.206623  [44864/60000]
loss: 2.197532  [51264/60000]
loss: 2.168028  [57664/60000]
Test Error: 
 Accuracy: 38.4%, Avg loss: 2.164087 

Epoch 2
-------------------------------
loss: 2.174839  [   64/60000]
loss: 2.164640  [ 6464/60000]
loss: 2.108209  [12864/60000]
loss: 2.126660  [19264/60000]
loss: 2.083283  [25664/60000]
loss: 2.020871  [32064/60000]
loss: 2.062218  [38464/60000]
loss: 1.984305  [44864/60000]
loss: 1.984343  [51264/60000]
loss: 1.922364  [57664/60000]
Test Error: 
 Accuracy: 56.1%, Avg loss: 1.916832 

Epoch 3
-------------------------------
loss: 1.948621  [   64/60000]
loss: 1.918276  [ 6464/60000]
loss: 1.802920  [12864/60000]
loss: 1.848468  [19264/60000]
loss: 1.737671  [25664/60000]
loss: 1.688171  [32064/600

In [ ]:

# save
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

# load
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))


In [16]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]



model.eval()
x, y = test_data[0][0], test_data[0][1]

# also with no autograd
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')



Predicted: "Ankle boot", Actual: "Ankle boot"
